In [23]:
import re

In [38]:
import pandas as pd
data = pd.read_csv(".\data\登記受付帳_22-23_築年数追加_前処理済み.csv")

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Barre\AppData\Local\Temp\ipykernel_19584\705551946.py:2: SyntaxWarning: invalid escape sequence '\d'
  data = pd.read_csv(".\data\登記受付帳_22-23_築年数追加_前処理済み.csv")


In [ ]:
data = pd.read_csv(".\data\登記受付帳22-23_築年数追加_登記原因26個.csv")

In [14]:
import pandas as pd
from tqdm import tqdm
import re
tqdm.pandas()  # tqdmをpandasで使用する設定

# CSVデータを読み込む
file_path = ".\data\登記受付帳22-23_築年数追加_登記原因26個.csv"
data = pd.read_csv(file_path)

# # 主要な登記原因のリスト
# valid_reasons = [
#     "所有権移転売買", "所有権移転相続・法人合併", "抹消登記", "抵当権の設定", 
#     "登記名義人の氏名等についての変更・更正", "所有権の保存(申請)", "表題", "滅失", 
#     "地目変更・更正", "所有権移転遺贈・贈与その他無償名義", "分筆", "権利", 
#     "変更・更正", "根抵当権の設定", "所有権移転その他の原因", "権利の移転(所有権を除く)", 
#     "地積変更・更正", "区分建物の表題", "仮登記(その他)", "処分の制限に関する登記", 
#     "合筆", "仮登記(所有権)", "所有権移転遺贈・贈与その他", "共同担保変更通知", 
#     "抹消登記/嘱託", "処分の制限に関する登記/嘱託"
# ]

# other_dict = {"登記名義人の氏名等についての変更更正": "登記名義人の氏名等についての変更・更正", "所有権移転相続法人合併": "所有権移転相続・法人合併",
#                "地目変更更正": "地目変更・更正", "所有権移転遺贈贈与その他無償名義": "所有権移転遺贈・贈与その他無償名義", "抹消登記嘱託": "抹消登記/嘱託",
#                  "処分の制限に関する登記嘱託": "処分の制限に関する登記/嘱託", "所有権の保存申請": "所有権の保存(申請)", "権利の移転所有権を除く": "権利の移転(所有権を除く)", "仮登記その他": "仮登記(その他)",
#                  "仮登記所有権": "仮登記(所有権)", "地積変更更正": "地積変更・更正"}


# # `reason`列を主要な登記原因で置き換える処理
# def map_reason(value):
#     for pre_reason in other_dict:
#         if pre_reason in value:
#             return other_dict[pre_reason]
        
#     for reason in valid_reasons:
#         if reason in value:
#             return reason
#     return value  # その他の場合は元の値をそのまま保持

# # tqdmを使って途中経過を表示しながら適用
# data['reason'] = data['reason'].astype(str).progress_apply(map_reason)

# # land_num列のハイフンを全角から半角に変換する処理を追加
# data['land_num'] = data['land_num'].astype(str).str.replace('－', '-', regex=False)



# 漢数字をアラビア数字に変換する辞書
kanji_to_num = {
    '一': '1', '二': '2', '三': '3', '四': '4', '五': '5', 
    '六': '6', '七': '7', '八': '8', '九': '9', '十': '10'
}

# 漢数字をアラビア数字に変換する関数
def convert_kanji_to_number(location):
    if pd.isnull(location):
        return location  # NaNはそのまま返す

    # 丁目の直前にある漢数字を変換
    def replace_kanji(match):
        kanji = match.group(1)  # キャプチャグループ1を取得
        return kanji_to_num.get(kanji, kanji) + '丁目'  # 漢数字をアラビア数字に変換して返す

    # 正規表現で "漢数字+丁目" を探し出して変換
    return re.sub(r'([一二三四五六七八九十])丁目', replace_kanji, location)

# 地番部分を抽出して、丁目・番地（番も含む）・号も処理する関数
def extract_land_num(location):
    match = re.search(
        r'(\d+)丁目(\d+[-－]\d+(?:[-－]\d+)?)(?:番地|番)?(?:号)?|'
        r'(\d+)丁目(\d+)(?:番地|番)(\d+)号|'
        r'(\d+)丁目(\d+)(?:番地|番)|'
        r'(\d+)丁目|'
        r'(\d+[-－]\d+(?:[-－]\d+)?)$', 
        str(location)
    )
    if match:
        if match.group(1) and match.group(2):  # "1丁目326-402" のような形式
            return f"{match.group(1)}-{match.group(2)}"
        elif match.group(3) and match.group(4) and match.group(5):  # "4丁目20番地8号" or "4丁目20番8号"
            return f"{match.group(3)}-{match.group(4)}-{match.group(5)}"
        elif match.group(6) and match.group(7):  # "4丁目20番地" or "4丁目20番"
            return f"{match.group(6)}-{match.group(7)}"
        elif match.group(8):  # "3丁目" のみ
            return f"{match.group(8)}"
        elif match.group(9):  # "30-1" のような形式
            return match.group(9)
    return None

# land_numに丁目情報を追加する関数
def update_land_num_with_chome(location, land_num):
    match = re.search(r'(\d+)丁目', location)
    if match:
        chome = match.group(1)  # 丁目部分を取得
        if pd.notnull(land_num):
            return f"{chome}-{land_num}"  # 丁目をland_numの先頭に追加
        else:
            return chome  # 丁目のみがland_numになる
    return land_num  # 丁目がなければそのまま

# location列に漢数字をアラビア数字に変換
data['location'] = data['location'].apply(convert_kanji_to_number)

# 地番部分を抽出してland_num列に更新
data['land_num_update'] = data['location'].apply(extract_land_num)
data['land_num'] = data['land_num'].fillna(data['land_num_update'])

# location列から地番部分を削除（番地、番、号、〇丁目も含めて削除）
data['location'] = data['location'].apply(
    lambda x: re.sub(
        r'(\d+)丁目(\d+[-－]\d+(?:[-－]\d+)?)(?:番地|番)?(?:号)?|'
        r'(\d+)丁目(\d+)(?:番地|番)(\d+)号|'
        r'(\d+)丁目(\d+)(?:番地|番)|'
        r'(\d+)丁目|'
        r'(\d+[-－]\d+(?:[-－]\d+)?)$', 
        '', 
        str(x)
    ).strip()
)

# 丁目情報をland_num列に追加
data['land_num'] = data.apply(lambda row: update_land_num_with_chome(row['location'], row['land_num']), axis=1)

# land_num_update列は確認用なので削除
data.drop(columns=['land_num_update'], inplace=True)


# register_date列をdatetime型に変換
data['register_date'] = pd.to_datetime(data['register_date'], errors='coerce')

# register_dateでソート
data = data.sort_values(by='register_date')



# 同一住所，同一日時，同一登記原因の登記を重複除去

# 前処理されたデータを保存する場合
data.to_csv(".\data\登記受付帳_22-23_築年数追加_前処理済み.csv", index=False)

# 結果確認用
# print(data['reason'].value_counts())


<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:137: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:137: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Barre\AppData\Local\Temp\ipykernel_25844\1172060106.py:7: SyntaxWarning: invalid escape sequence '\d'
  file_path = ".\data\登記受付帳22-23_築年数追加_登記原因26個.csv"
C:\Users\Barre\AppData\Local\Temp\ipykernel_25844\1172060106.py:137: SyntaxWarning: invalid escape sequence '\d'
  data.to_csv(".\data\登記受付帳_22-23_築年数追加_前処理済み.csv", index=False)
C:\Users\Barre\AppData\Local\Temp\ipykernel_25844\1172060106.py:8: DtypeWarning: Columns (16,21,22,25,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [15]:
data[(data["pref"] == "鹿児島県") & (data["location"] == "鹿屋市札元2丁目") & (data["land_num"] == "3718-2")]

,register_num,register_date,register_type,reason,pref,city,new_or_existing,type,location,land_num,...,parking_spaces,main_direction,land_area,building_area,total_floor_area,max_layout,min_layout,property_type,structure,building_age


In [16]:
data[(data["pref"] == "鹿児島県") & (data["location"] == "鹿屋市札元") & (data["land_num"] == "3718-2")]

,register_num,register_date,register_type,reason,pref,city,new_or_existing,type,location,land_num,...,parking_spaces,main_direction,land_area,building_area,total_floor_area,max_layout,min_layout,property_type,structure,building_age
12637515,2495,2022-02-25,単独,所有権移転売買,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12637517,5105,2022-04-22,単独,地目変更・更正,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12637518,5104,2022-04-22,単独,分筆,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12637516,12384,2022-10-13,連続,所有権移転売買,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5630479,3521,2023-03-23,連続,所有権移転売買,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5630480,4276,2023-04-07,連続,所有権移転売買,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5630481,14528,2023-11-06,連続,所有権移転売買,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5630482,15376,2023-11-22,連続,所有権移転売買,鹿児島県,鹿屋市,既,土地,鹿屋市札元,3718-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
import pandas as pd
from tqdm import tqdm
import re
tqdm.pandas()  # tqdmをpandasで使用する設定

# CSVデータを読み込む
# file_path = ".\data\登記受付帳_22-23_築年数追加.csv"
# data = pd.read_csv(file_path)

# 主要な登記原因のリスト
valid_reasons = [
    "所有権移転売買", "所有権移転相続・法人合併", "抹消登記", "抵当権の設定", 
    "登記名義人の氏名等についての変更・更正", "所有権の保存(申請)", "表題", "滅失", 
    "地目変更・更正", "所有権移転遺贈・贈与その他無償名義", "分筆", "権利", 
    "変更・更正", "根抵当権の設定", "所有権移転その他の原因", "権利の移転(所有権を除く)", 
    "地積変更・更正", "区分建物の表題", "仮登記(その他)", "処分の制限に関する登記", 
    "合筆", "仮登記(所有権)", "所有権移転遺贈・贈与その他", "共同担保変更通知", 
    "抹消登記/嘱託", "処分の制限に関する登記/嘱託"
]

other_dict = {"登記名義人の氏名等についての変更更正": "登記名義人の氏名等についての変更・更正", "所有権移転相続法人合併": "所有権移転相続・法人合併",
               "地目変更更正": "地目変更・更正", "所有権移転遺贈贈与その他無償名義": "所有権移転遺贈・贈与その他無償名義", "抹消登記嘱託": "抹消登記/嘱託",
                 "処分の制限に関する登記嘱託": "処分の制限に関する登記/嘱託", "所有権の保存申請": "所有権の保存(申請)", "権利の移転所有権を除く": "権利の移転(所有権を除く)", "仮登記その他": "仮登記(その他)",
                 "仮登記所有権": "仮登記(所有権)", "地積変更更正": "地積変更・更正"}


# `reason`列を主要な登記原因で置き換える処理
def map_reason(value):
    for pre_reason in other_dict:
        if pre_reason in value:
            return other_dict[pre_reason]
        
    for reason in valid_reasons:
        if reason in value:
            return reason
    return value  # その他の場合は元の値をそのまま保持

# tqdmを使って途中経過を表示しながら適用
data['reason'] = data['reason'].astype(str).progress_apply(map_reason)

100%|██████████| 17196215/17196215 [00:13<00:00, 1293279.45it/s]


In [39]:
data.head()

,register_num,register_date,register_type,reason,pref,city,new_or_existing,type,location,land_num,...,parking_spaces,main_direction,land_area,building_area,total_floor_area,max_layout,min_layout,property_type,structure,building_age
0,1900,2022-01-01,連続,分筆,岐阜県,関市,既,土地,関市洞戸尾倉,288-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1613,2022-01-01,単独,抹消登記,福島県,福島市,既,土地,福島市宮代字安太渕30-1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16188,2022-01-01,単独,所有権移転売買,福島県,郡山市,既,土地,郡山市麓山1丁目326-402,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1113,2022-01-01,連先,所有権移転相続・法人合併,福島県,福島市,既,土地,福島市瀬上町字東町1丁目6-3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1118,2022-01-01,単独,所有権移転売買,福島県,福島市,既,土地,福島市佐倉下字台18-5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# 漢数字をアラビア数字に変換する辞書
kanji_to_num = {
    '一': '1', '二': '2', '三': '3', '四': '4', '五': '5', 
    '六': '6', '七': '7', '八': '8', '九': '9', '十': '10'
}

# 漢数字をアラビア数字に変換する関数
def convert_kanji_to_number(location):
    if pd.isnull(location):
        return location  # NaNはそのまま返す

    # 丁目の直前にある漢数字を変換
    def replace_kanji(match):
        kanji = match.group(1)  # キャプチャグループ1を取得
        return kanji_to_num.get(kanji, kanji) + '丁目'  # 漢数字をアラビア数字に変換して返す

    # 正規表現で "漢数字+丁目" を探し出して変換
    return re.sub(r'([一二三四五六七八九十])丁目', replace_kanji, location)

# location列に適用
data['location'] = data['location'].apply(convert_kanji_to_number)

In [30]:
# 地番部分を抽出して、丁目・番地（番も含む）・号も処理する関数
def extract_land_num(location):
    match = re.search(
        r'(\d+)丁目(\d+[-－]\d+(?:[-－]\d+)?)(?:番地|番)?(?:号)?|'
        r'(\d+)丁目(\d+)(?:番地|番)(\d+)号|'
        r'(\d+)丁目(\d+)(?:番地|番)|'
        r'(\d+)丁目|'
        r'(\d+[-－]\d+(?:[-－]\d+)?)$', 
        str(location)
    )
    if match:
        if match.group(1) and match.group(2):  # "1丁目326-402" のような形式
            return f"{match.group(1)}-{match.group(2)}"
        elif match.group(3) and match.group(4) and match.group(5):  # "4丁目20番地8号" or "4丁目20番8号"
            return f"{match.group(3)}-{match.group(4)}-{match.group(5)}"
        elif match.group(6) and match.group(7):  # "4丁目20番地" or "4丁目20番"
            return f"{match.group(6)}-{match.group(7)}"
        elif match.group(8):  # "3丁目" のみ
            return f"{match.group(8)}"
        elif match.group(9):  # "30-1" のような形式
            return match.group(9)
    return None

# land_numに移動し、locationから削除
data['land_num_update'] = data['location'].apply(extract_land_num)
data['land_num'] = data['land_num'].fillna(data['land_num_update'])

# location列から地番部分を削除（番地、番、号、〇丁目も含めて削除）
data['location'] = data['location'].apply(
    lambda x: re.sub(
        r'(\d+)丁目(\d+[-－]\d+(?:[-－]\d+)?)(?:番地|番)?(?:号)?|'
        r'(\d+)丁目(\d+)(?:番地|番)(\d+)号|'
        r'(\d+)丁目(\d+)(?:番地|番)|'
        r'(\d+)丁目|'
        r'(\d+[-－]\d+(?:[-－]\d+)?)$', 
        '', 
        str(x)
    ).strip()
)

# land_num_update列は確認用なので削除
data.drop(columns=['land_num_update'], inplace=True)

# 結果を確認
data.head()


,register_num,register_date,register_type,reason,pref,city,new_or_existing,type,location,land_num,...,parking_spaces,main_direction,land_area,building_area,total_floor_area,max_layout,min_layout,property_type,structure,building_age
0,1900,2022-01-01,連続,分筆,岐阜県,関市,既,土地,関市洞戸尾倉,288-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1613,2022-01-01,単独,抹消登記,福島県,福島市,既,土地,福島市宮代字安太渕,30-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16188,2022-01-01,単独,所有権移転売買,福島県,郡山市,既,土地,郡山市麓山,1-326-402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1113,2022-01-01,連先,所有権移転相続・法人合併,福島県,福島市,既,土地,福島市瀬上町字東町,1-6-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1118,2022-01-01,単独,所有権移転売買,福島県,福島市,既,土地,福島市佐倉下字台,18-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
data.tail()

,register_num,register_date,register_type,reason,pref,city,new_or_existing,type,location,land_num,...,parking_spaces,main_direction,land_area,building_area,total_floor_area,max_layout,min_layout,property_type,structure,building_age
17196210,6037,2023-12-28,単独,所有権移転売買,兵庫県,三田市,既,土地,三田市上相野,1143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17196211,79255,2023-12-28,連続,所有権の保存(申請),宮城県,仙台市青葉区,既,建物,仙台市青葉区上愛子字平治,31-89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17196212,79256,2023-12-28,連続,抵当権の設定,宮城県,仙台市青葉区,既,建物,仙台市青葉区上愛子字平治,31-89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17196213,90700,2023-12-28,連続,所有権移転相続・法人合併,愛知県,名古屋市昭和区,既,土地,名古屋市昭和区車田町,48-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17196214,27097,2023-12-28,連続,登記名義人の氏名等についての変更・更正,栃木県,大田原市,既,土地,大田原市野崎,6-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
len(data["reason"].unique())

4838

In [33]:
# # register_date列をdatetime型に変換
# data['register_date'] = pd.to_datetime(data['register_date'], errors='coerce')

# # register_dateでソート
# data = data.sort_values(by='register_date')

# 同一住所，同一日時，同一登記原因の登記を重複除去

# 前処理されたデータを保存する場合
data.to_csv(".\data\登記受付帳_22-23_築年数追加_前処理済み.csv", index=False)

<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Barre\AppData\Local\Temp\ipykernel_11268\2744054394.py:10: SyntaxWarning: invalid escape sequence '\d'
  data.to_csv(".\data\登記受付帳_22-23_築年数追加_前処理済み.csv", index=False)
